In [1]:
import sys
import os
import signal

from isaacsim import SimulationApp

CONFIG = {
  'headless': False,
  'display_options': 3286
}

# Example for creating a RTX lidar sensor and publishing PointCloud2 data
simulation_app = SimulationApp(CONFIG)
simulation_app.update()

spinning = True
def sigint_handler(sig, frame):
  global spinning
  spinning = False
signal.signal(signal.SIGINT, sigint_handler)

kernel = get_ipython().kernel
kernel.pre_handler_hook = lambda: None
kernel.post_handler_hook = lambda: None

def spin(update_function, *args, **kwargs):
  global spinning
  print('gui can be used now. interrupt kernel to go back to notebook')
  while spinning:
    update_function(*args, **kwargs)
  spinning = True

def resume_gui():
  spin(lambda: simulation_app.update())
  
if CONFIG['headless']:
  del spin
  del resume_gui
  

print('workdir:', os.getcwd())


Starting kit application with the following args:  ['/f/ov/pkg/isaac-sim-4.1.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/f/ov/pkg/isaac-sim-4.1.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/f/ov/pkg/isaac-sim-4.1.0/kit', '--/persistent/app/viewport/displayOptions=3286', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/exts', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/apps', '--/physics/cudaDevice=0', '--portable']
Passing the following args to the base kit application:  ['-f', '/home/linh/.local/share/jupyter/runtime/kernel-61f326ca-96f4-41eb-9bb0-b01c32bf749d.json']
[Warning] [omni.isaac.kit.simulation_app] Non-headless mode not supported with jupyt

Warp 1.2.1 initialized:
   CUDA Toolkit 11.8, Driver 12.5
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 4060" (8 GiB, sm_89, mempool enabled)
   Kernel cache:
     /home/linh/.cache/warp/1.2.1
] [ext: omni.kit.widget.text_editor-1.0.2] startup
[2.996s] [ext: omni.kit.viewport.window-106.0.8] startup
[3.019s] [ext: omni.kit.window.property-1.11.1] startup
[3.021s] [ext: omni.usdphysics-106.0.20] startup
[3.025s] [ext: omni.graph.tools-1.78.0] startup
[3.049s] [ext: omni.kit.viewport.utility-1.0.17] startup
[3.050s] [ext: omni.kit.property.usd-3.21.28] startup
[3.060s] [ext: omni.graph-1.135.0] startup
[3.190s] [ext: omni.graph.image.core-0.3.2] startup
[3.195s] [ext: omni.graph.ui-1.70.0] startup
[3.213s] [ext: omni.graph.action_core-1.1.4] startup
[3.224s] [ext: omni.kit.window.cursor-1.1.2] startup
[3.225s] [ext: omni.graph.image.nodes-1.0.2] startup
[3.228s] [ext: omni.graph.action_nodes-1.23.0] startup
[3.239s] [ext: omni.kit.viewport.menubar.core-106.

In [2]:
import os
import glob
import json
import copy

import carb
import omni
from pxr import Gf, Sdf

from omni.isaac.core import World
from omni.isaac.core.objects import *
from omni.isaac.core.utils.stage import *
from omni.isaac.core.utils.rotations import *
from omni.isaac.sensor import LidarRtx
import omni.replicator.core as rep

import numpy as np
import matplotlib.pyplot as plt
import psutil

proc = psutil.Process()
meminfo = proc.memory_info()
start_mem = meminfo.rss / 1024 / 1024

In [3]:
def setup_scene():
    create_new_stage()
    world = World(physics_dt=0, rendering_dt=1.0/30)
    world.scene.add_default_ground_plane()
    lidar = LidarRtx(
        prim_path='/lidar',
        name='lidar',
        config_file_name='OS1_REV6_32ch10hz512res_noiseless',
        translation=[0, 0, 10],
        orientation=euler_angles_to_quat([0, 90, 0], degrees=True),
    )
    lidar.add_point_cloud_data_to_frame()
    lidar.add_azimuth_data_to_frame()
    lidar.add_elevation_data_to_frame()
    lidar.add_intensities_data_to_frame()
    lidar.add_range_data_to_frame()
    
    world.scene.add(lidar)
    world.reset()
    return world, lidar

world, lidar = setup_scene()

In [4]:
for _ in range(1000):
    world.step()
    frame = lidar.get_current_frame()
    
meminfo = proc.memory_info()
current_mem = meminfo.rss / 1024 / 1024
print(f'Increased: {current_mem - start_mem:.2f} MB')


Increased: 171.93 MB


In [5]:
import omni.graph.core as og

def _og_connect(no, ao, ni, ai):
    og.Controller.connect(f'{no}.outputs:{ao}', f'{ni}.inputs:{ai}')
    
def get_mem():
    meminfo = proc.memory_info()
    return meminfo.rss / 1024 / 1024

world, lidar = setup_scene()
rss = get_mem()

lidar_node = lidar._point_cloud_annotator.get_node().get_prim_path()
graph_path = '/'.join(lidar_node.split('/')[:-1])
event_node = f'{graph_path}/event_node'
event_name = 'rtxLidarReady'
og.Controller.create_node(
    event_node, "omni.graph.action.SendMessageBusEvent")
og.Controller.attribute("inputs:eventName", event_node).set(event_name)
_og_connect(lidar_node, "exec", event_node, "execIn")
attrs = [("azimuth", "float[]"),
            ("elevation", "float[]"),
            ("range", "float[]"),
            ("transform", "matrixd[4]")]
for aname, atype in attrs:
    og.Controller.create_attribute(event_node, aname, atype)
    _og_connect(lidar_node, aname, event_node, aname)
    
for _ in range(1000):
    world.step()
    frame = lidar.get_current_frame()
    
print(f'Increased: {get_mem() - rss:.2f} MB')

Increased: 2073.61 MB


In [6]:
resume_gui()

gui can be used now. interrupt kernel to go back to notebook


In [11]:
event_type = carb.events.type_from_string(event_name)
message_bus = omni.kit.app.get_app().get_message_bus_event_stream()
print(message_bus.event_count)


0
